1. Read in CMIP6 model data 
2. Read in ETCCDI data 
3. Make sure on same grid, time etc. 
4. Calculate composites for variables (take multi model mean at end)
5. Sign comparison between mean and extreme for each variable (for mmm)

In [1]:
# 0. Imports 
import xarray as xr
import numpy as np
import sys
sys.path.append('/home/563/rl5183/PhD-research/Functions')
import functions as func
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import dask
dask.config.set({'array.slicing.split_large_chunks': False})

In [2]:
# 1. Read in CMIP6 model data 
data = xr.open_dataarray('/g/data/w40/rl5183/CMIP6_20_ts_quad_detrend_1900_2015.nc')

In [3]:
# Calculate Nino3.4 index
nino34_list = []
for m in data.model:
    nino34_list.append(func.nino34_index(data.sel(model=m), '1950-01', '1979-12'))
nino34 = xr.DataArray(nino34_list, dims=['model', 'time'], coords=[data.model, data.time])

In [4]:
%%time
elnino_list_ts = []
lanina_list_ts = []
for m in nino34.model:
    el, la = func.sst_comp(nino34.sel(model=m), data.sel(model=m), 'DJF', 0.5, 6)
    elnino_list_ts.append(el)
    lanina_list_ts.append(la)
    
elnino_ts = xr.concat(elnino_list_ts, dim='model').chunk('auto')
lanina_ts = xr.concat(lanina_list_ts, dim='model').chunk('auto')

CPU times: user 15min 38s, sys: 5min 34s, total: 21min 13s
Wall time: 44min 3s


In [5]:
elnino_ts.to_netcdf('elnino_comp_ts_cmip6_20.nc')
lanina_ts.to_netcdf('lanina_comp_ts_cmip6_20.nc')

In [6]:
elnino_ts = xr.open_dataarray('elnino_comp_ts_cmip6_20.nc')
lanina_ts = xr.open_dataarray('lanina_comp_ts_cmip6_20.nc')